In [12]:
import os
import csv
from collections import Counter

os.getcwd()

'/home/maxjo/Projects/Bamboo_kaggle'

In [19]:
with open('./datasets/train.csv', encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)
    train_data = list(csv_reader)

train_data_fname, train_data_label = zip(*( (d['filename'], d['label']) for d in train_data ))

with open('./datasets/test.csv', encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)
    test_data = list(csv_reader)

test_data_fname = [d['filename'] for d in test_data]

In [20]:
len(train_data_fname), len(test_data_fname)

(5199, 1300)

In [15]:
butterfly_counts = Counter(train_data_label)

print(len(butterfly_counts))
print(butterfly_counts.most_common(10))

75
[('MOURNING CLOAK', 105), ('SLEEPY ORANGE', 85), ('ATALA', 80), ('BROWN SIPROETA', 79), ('MILBERTS TORTOISESHELL', 77), ('CRECENT', 77), ('ORANGE TIP', 77), ('BANDED ORANGE HELICONIAN', 77), ('SCARCE SWALLOW', 77), ('RED CRACKER', 77)]


In [22]:
display(butterfly_counts)

Counter({'MOURNING CLOAK': 105,
         'SLEEPY ORANGE': 85,
         'ATALA': 80,
         'BROWN SIPROETA': 79,
         'MILBERTS TORTOISESHELL': 77,
         'CRECENT': 77,
         'ORANGE TIP': 77,
         'BANDED ORANGE HELICONIAN': 77,
         'SCARCE SWALLOW': 77,
         'RED CRACKER': 77,
         'EASTERN PINE ELFIN': 76,
         'CHECQUERED SKIPPER': 76,
         'IPHICLUS SISTER': 76,
         'GREAT JAY': 75,
         'DANAID EGGFLY': 75,
         'COPPER TAIL': 75,
         'CLEOPATRA': 74,
         'EASTERN COMA': 74,
         'PURPLISH COPPER': 74,
         'EASTERN DAPPLE WHITE': 74,
         'CLOUDED SULPHUR': 74,
         'PAPER KITE': 72,
         'APPOLLO': 72,
         'CABBAGE WHITE': 72,
         'SOOTYWING': 72,
         'COMMON WOOD-NYMPH': 72,
         'MONARCH': 72,
         'RED POSTMAN': 71,
         'MANGROVE SKIPPER': 70,
         'STRAITED QUEEN': 70,
         'CLODIUS PARNASSIAN': 70,
         'ADONIS': 70,
         'ORANGE OAKLEAF': 70,
       